In [1]:
import pandas as pd
import numpy as np

# set up connection to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# import dataframe
path = '/content/drive/My Drive/Colab Notebooks/Insurance_Investigation/cleaned_insurance_complaint_data.xlsx'
insurance = pd.read_excel(path)

In [6]:
# filter out dates prior to covid and other companies
insurance = insurance.loc[insurance['Complaint_filed_against'] == 'BCBS']
insurance = insurance.loc[insurance['Received_date'] >= '5/1/2020']

In [9]:
# save reduced table for tableau
ready = insurance

ready.to_excel('tableau_insurance_complaint_data.xlsx')
!cp tableau_insurance_complaint_data.xlsx "/content/drive/My Drive/Colab Notebooks/Insurance_Investigation/"

In [39]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2276 entries, 54571 to 60261
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Unnamed: 0               2276 non-null   int64         
 1   Complaint_filed_against  2276 non-null   object        
 2   Complaint_filed_by       2276 non-null   object        
 3   Reason_complaint_filed   2276 non-null   object        
 4   Confirmed_complaint      2276 non-null   object        
 5   Received_date            2276 non-null   datetime64[ns]
 6   Closed_date              2276 non-null   datetime64[ns]
 7   Complaint_type           2276 non-null   object        
 8   Coverage_level           2276 non-null   object        
 9   Complainant_type         2276 non-null   object        
 10  Main_Reason              2276 non-null   object        
 11  Sub_Reason               2276 non-null   object        
 12  Num_Stages               2276

In [38]:
from itertools import count
# Time Graph table
sa = insurance[['Received_date','Confirmed_complaint', 'Main_Reason']]

sa = pd.pivot_table(sa, index = 'Received_date', columns = 'Confirmed_complaint', values = 'Main_Reason', aggfunc = 'count')

# fill in missing dates with 0
idx = pd.date_range('2020-05-01','2023-03-29')
sa = sa.reindex(idx, fill_value=0)
sa.reset_index()

# Get totals
sa = sa.fillna(0)
sa['Total'] = sa.sum(axis = 1)

In [ ]:
# Save Time graph in drive
sa.to_excel('timegraph_insurance_complaint_data.xlsx')
!cp timegraph_insurance_complaint_data.xlsx "/content/drive/My Drive/Colab Notebooks/Insurance_Investigation/"

In [68]:
## Survival Analysis with Claim Length
surv = insurance[['Claim_Length', 'Confirmed_complaint', 'Main_Reason']]
surv = pd.pivot_table(surv, index = 'Claim_Length', columns = 'Confirmed_complaint', values = 'Main_Reason', aggfunc = 'count')
m = insurance['Claim_Length'].max()

# Fill in missings Number of Stages
nums = [i for i in range(0,m+1)]
surv = surv.reindex(nums, fill_value=0)
surv.reset_index()

# Get running totals
surv = surv.fillna(0)
surv['Total Yes'] = surv.loc[::-1, 'Yes'].cumsum()[::-1]
surv['Total No'] = surv.loc[::-1, 'No'].cumsum()[::-1]
surv['Perc_Yes'] = surv['Total Yes']/(surv.iloc[0]['Total Yes'])
surv['Perc_No'] = surv['Total No']/(surv.iloc[0]['Total No'])

In [70]:
# Save Survival Analysis with Claim Length in drive
surv.to_excel('survival_insurance_complaint_data.xlsx')
!cp survival_insurance_complaint_data.xlsx "/content/drive/My Drive/Colab Notebooks/Insurance_Investigation/"

In [73]:
## Survival Analysis with Number of Stages
survs = insurance[['Num_Stages', 'Confirmed_complaint', 'Main_Reason']]
survs = pd.pivot_table(survs, index = 'Num_Stages', columns = 'Confirmed_complaint', values = 'Main_Reason', aggfunc = 'count')
mm = insurance['Num_Stages'].max()

# Fill in missings Number of Stages
numss = [i for i in range(0,mm+1)]
survs = survs.reindex(numss, fill_value=0)
survs.reset_index()

# Get running totals
survs = survs.fillna(0)
survs['Total Yes'] = survs.loc[::-1, 'Yes'].cumsum()[::-1]
survs['Total No'] = survs.loc[::-1, 'No'].cumsum()[::-1]
survs['Perc_Yes'] = survs['Total Yes']/(survs.iloc[0]['Total Yes'])
survs['Perc_No'] = survs['Total No']/(survs.iloc[0]['Total No'])

In [76]:
# Save Survival Analysis with Number of Stages in drive
survs.to_excel('survival_stag_insurance_complaint_data.xlsx')
!cp survival_stag_insurance_complaint_data.xlsx "/content/drive/My Drive/Colab Notebooks/Insurance_Investigation/"

In [ ]:
## Calculate Percents of Top Main reasons

# Get Reasons with most claims
tops = insurance['Main_Reason'].value_counts().nlargest(7)
names = tops.index

# Get only entries with one of top reasons.
perc = insurance.loc[insurance['Main_Reason'].isin(names)]
perc = perc[['Main_Reason', 'Confirmed_complaint', 'Complaint_filed_against']]

# Get Percentages of top reasons
p = pd.pivot_table(perc, index = 'Main_Reason', columns = 'Confirmed_complaint', values = 'Complaint_filed_against', aggfunc = 'count')
p = p.fillna(0)
p['Total'] = p.sum(axis=1)
p['Perc_Yes'] = p['Yes']/p['Total']
p['Perc_No'] = 1 - p['Perc_Yes']
p.reset_index()

In [126]:
# Save Percent in drive
p.to_excel('perc_insurance_complaint_data.xlsx')
!cp perc_insurance_complaint_data.xlsx "/content/drive/My Drive/Colab Notebooks/Insurance_Investigation/"